In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
from pylab import rcParams

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import recall_score, classification_report, auc, roc_curve
from sklearn.metrics import precision_recall_fscore_support, f1_score

from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from numpy import mean

from numpy.random import seed
seed(7)

from sklearn.model_selection import train_test_split

SEED = 123 #used to help randomly select the data points
DATA_SPLIT_PCT = 0.2

rcParams['figure.figsize'] = 8, 6
LABELS = ["Normal","Break"]

In [2]:
df = pd.read_csv("processed_data18.csv") 
df.head(n=5)  # visualize the data.

,Unnamed: 0,time,y,x1,x2,x3,x4,x5,x6,x7,...,trend_x52,trend_x53,trend_x54,trend_x55,trend_x56,trend_x57,trend_x58,trend_x59,trend_x60,trend_x61
0,0,5/1/99 0:00,0,0.376665,-4.596435,-4.095756,13.497687,-0.118830,-20.669883,0.000732,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,5/1/99 0:02,0,0.475720,-4.542502,-4.018359,16.230659,-0.128733,-18.758079,0.000732,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,5/1/99 0:04,0,0.363848,-4.681394,-4.353147,14.127997,-0.138636,-17.836632,0.010803,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,5/1/99 0:06,0,0.301590,-4.758934,-4.023612,13.161566,-0.148142,-18.517601,0.002075,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5/1/99 0:08,0,0.265578,-4.749928,-4.333150,15.267340,-0.155314,-17.505913,0.000732,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
index_lower_limit = 4177
index_upper_limit = 11503

df = pd.DataFrame(df[index_lower_limit: index_upper_limit])
df

,Unnamed: 0,time,y,x1,x2,x3,x4,x5,x6,x7,...,trend_x52,trend_x53,trend_x54,trend_x55,trend_x56,trend_x57,trend_x58,trend_x59,trend_x60,trend_x61
4177,4177,5/7/99 13:28,1,-3.787279,-8.936167,-9.729623,-214.34934,0.062673,-125.748970,-0.429273,...,-6.641181,-6.780132,2.726780,2.659396,23.805840,-2.535488,0.025905,0.569830,0.005335,0.000000
4178,4178,5/7/99 13:34,0,-1.632907,8.798339,-5.254108,-176.88787,-0.086024,-86.578254,-0.429273,...,-6.662573,-6.790757,2.729129,3.641574,20.196577,-2.554009,0.025694,0.587876,0.005263,0.000000
4179,4179,5/7/99 13:36,0,-2.027379,8.666358,5.840922,-179.27187,-0.086024,-77.143959,-0.076540,...,-6.682887,-6.800581,2.731489,5.316500,16.569363,-2.545590,0.025492,0.635080,0.005190,0.000000
4180,4180,5/7/99 13:38,0,-0.619546,6.910144,7.031393,-177.13259,-0.086024,-73.890693,-0.069155,...,-6.702123,-6.809606,2.733862,6.060106,12.452740,-2.541381,0.025302,0.617112,0.005118,0.000000
4181,4181,5/7/99 13:40,0,-1.019356,7.319770,7.415930,-179.34461,-0.098994,-72.566185,-0.069155,...,-6.720291,-6.817831,2.736246,7.746066,8.838936,-2.521384,0.025121,0.572913,0.005045,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11498,11498,5/18/99 4:14,0,0.257948,-4.848939,3.191914,-322.78161,-0.671366,-279.408440,0.050782,...,-0.952847,9.259511,3.039350,74.394722,-73.641466,0.507460,-0.102175,0.527703,0.007121,0.034483
11499,11499,5/18/99 4:16,0,0.346350,-4.752980,3.282870,-322.78161,-0.701014,-279.408440,0.054475,...,-1.460592,9.270750,3.035607,76.489491,-76.150706,0.403172,-0.097211,0.543292,0.007431,0.034483
11500,11500,5/18/99 4:18,0,0.518950,-4.811716,3.433512,-322.78161,-0.445338,-279.408440,0.060853,...,-1.966536,9.282157,3.031982,78.689686,-78.845924,0.241003,-0.092252,0.545655,0.007743,0.034483
11501,11501,5/18/99 4:20,0,0.810028,-4.795506,3.363568,-322.78161,-0.174662,-279.408440,0.060853,...,-2.470678,9.293741,3.028475,81.129232,-81.428658,0.111563,-0.087297,0.544319,0.008056,0.034483


In [4]:
df['y'].value_counts()[1]


38

In [5]:
df['y'].value_counts()[0]


7288

In [6]:
sign = lambda x: (1, -1)[x < 0]

def curve_shift(df, shift_by):
    '''
    This function will shift the binary labels in a dataframe.
    The curve shift will be with respect to the 1s. 
    For example, if shift is -2, the following process
    will happen: if row n is labeled as 1, then
    - Make row (n+shift_by):(n+shift_by-1) = 1.
    - Remove row n.
    i.e. the labels will be shifted up to 2 rows up.
    
    Inputs:
    df       A pandas dataframe with a binary labeled column. 
             This labeled column should be named as 'y'.
    shift_by An integer denoting the number of rows to shift.
    
    Output
    df       A dataframe with the binary labels shifted by shift.
    '''

    vector = df['y'].copy()
    for s in range(abs(shift_by)):
        tmp = vector.shift(sign(shift_by))
        tmp = tmp.fillna(0)
        vector += tmp
    labelcol = 'y'
    # Add vector to the df
    df.insert(loc=0, column=labelcol+'tmp', value=vector)
    # Remove the rows with labelcol == 1.
    df = df.drop(df[df[labelcol] == 1].index)
    # Drop labelcol and rename the tmp col as labelcol
    df = df.drop(labelcol, axis=1)
    df = df.rename(columns={labelcol+'tmp': labelcol})
    # Make the labelcol binary
    df.loc[df[labelcol] > 0, labelcol] = 1

    return df

In [7]:
'''
Shift the data by 2 units, equal to 4 minutes.

Test: Testing whether the shift happened correctly.
'''
print('Before shifting')  # Positive labeled rows before shifting.
one_indexes = df.index[df['y'] == 1]
display(df.iloc[(one_indexes[0]-3):(one_indexes[0]+2), 0:5].head(n=5))

# Shift the response column y by 2 rows to do a 4-min ahead prediction.
df = curve_shift(df, shift_by = -2)

print('After shifting')  # Validating if the shift happened correctly.
display(df.iloc[(one_indexes[0]-4):(one_indexes[0]+1), 0:5].head(n=5))  

Before shifting


,Unnamed: 0,time,y,x1,x2
8351,8351,5/13/99 14:06,0,1.320275,0.863326
8352,8352,5/13/99 14:08,0,1.301173,0.752034
8353,8353,5/13/99 14:10,0,1.319648,0.628083
8354,8354,5/13/99 14:12,0,1.304881,0.643592
8355,8355,5/13/99 14:14,0,1.313052,0.590009


After shifting


,y,Unnamed: 0,time,x1,x2
8369,0.0,8369,5/13/99 14:42,1.301987,0.509212
8370,0.0,8370,5/13/99 14:44,1.222639,0.484897
8371,0.0,8371,5/13/99 14:46,1.202801,0.554038
8372,0.0,8372,5/13/99 14:48,1.253957,0.661002
8373,0.0,8373,5/13/99 14:50,1.275130,0.374429


In [8]:
# Remove time column, and the categorical columns
df = df.drop(['time', 'x28', 'x61'], axis=1)

In [9]:
X = df.drop(['y'], axis=1)

y = df['y']

In [10]:
X = X.drop(['Unnamed: 0'], axis=1)
X

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,trend_x52,trend_x53,trend_x54,trend_x55,trend_x56,trend_x57,trend_x58,trend_x59,trend_x60,trend_x61
4178,-1.632907,8.798339,-5.254108,-176.88787,-0.086024,-86.578254,-0.429273,-0.085388,-0.120087,0.110624,...,-6.662573,-6.790757,2.729129,3.641574,20.196577,-2.554009,0.025694,0.587876,0.005263,0.000000
4179,-2.027379,8.666358,5.840922,-179.27187,-0.086024,-77.143959,-0.076540,-0.071186,0.363245,0.141565,...,-6.682887,-6.800581,2.731489,5.316500,16.569363,-2.545590,0.025492,0.635080,0.005190,0.000000
4180,-0.619546,6.910144,7.031393,-177.13259,-0.086024,-73.890693,-0.069155,-0.071186,0.429857,0.171341,...,-6.702123,-6.809606,2.733862,6.060106,12.452740,-2.541381,0.025302,0.617112,0.005118,0.000000
4181,-1.019356,7.319770,7.415930,-179.34461,-0.098994,-72.566185,-0.069155,-0.071186,0.419785,0.270206,...,-6.720291,-6.817831,2.736246,7.746066,8.838936,-2.521384,0.025121,0.572913,0.005045,0.000000
4182,-1.232371,7.560638,6.867594,-175.63545,-0.150874,-69.186195,-0.069155,-0.071186,0.410368,0.296584,...,-6.737389,-6.825256,2.738643,9.229426,5.216208,-2.525383,0.024950,0.587208,0.004973,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11498,0.257948,-4.848939,3.191914,-322.78161,-0.671366,-279.408440,0.050782,0.058823,0.009922,0.031699,...,-0.952847,9.259511,3.039350,74.394722,-73.641466,0.507460,-0.102175,0.527703,0.007121,0.034483
11499,0.346350,-4.752980,3.282870,-322.78161,-0.701014,-279.408440,0.054475,0.048752,0.009922,0.031699,...,-1.460592,9.270750,3.035607,76.489491,-76.150706,0.403172,-0.097211,0.543292,0.007431,0.034483
11500,0.518950,-4.811716,3.433512,-322.78161,-0.445338,-279.408440,0.060853,0.058823,0.007404,0.021628,...,-1.966536,9.282157,3.031982,78.689686,-78.845924,0.241003,-0.092252,0.545655,0.007743,0.034483
11501,0.810028,-4.795506,3.363568,-322.78161,-0.174662,-279.408440,0.060853,0.049759,0.008243,0.031699,...,-2.470678,9.293741,3.028475,81.129232,-81.428658,0.111563,-0.087297,0.544319,0.008056,0.034483


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_train.shape, X_test.shape

((5830, 607), (1458, 607))

In [12]:
y_train

7711     0.0
10700    0.0
7567     0.0
9295     0.0
11347    0.0
        ... 
9130     0.0
7450     0.0
5834     0.0
6789     0.0
6915     0.0
Name: y, Length: 5830, dtype: float64

In [13]:
# Feature scaling

In [14]:
cols = X_train.columns

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = pd.DataFrame(X_train, columns=[cols])
X_test = pd.DataFrame(X_test, columns=[cols])


In [15]:

# define model
model = XGBClassifier()
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
# summarize performance
print('Mean ROC AUC: %.5f' % mean(scores))

Mean ROC AUC: 0.97409


In [16]:
model = XGBClassifier()

clf_0 = model.fit(X_train, y_train)

In [17]:
pred_y_0 = clf_0.predict(X_test)

print(accuracy_score(pred_y_0, y_test))

0.9917695473251029


In [18]:
target_names = ['class 0', 'class 1']
print(classification_report(y_test, pred_y_0, target_names=target_names))

# View confusion matrix for test data and predictions
confusion_matrix(y_test, pred_y_0)

              precision    recall  f1-score   support

     class 0       0.99      1.00      1.00      1442
     class 1       1.00      0.25      0.40        16

    accuracy                           0.99      1458
   macro avg       1.00      0.62      0.70      1458
weighted avg       0.99      0.99      0.99      1458



array([[1442,    0],
       [  12,    4]])

# Weighted XGBoost for Class Imbalance


In [19]:
# define model
#scale_pos_weight=total_negative_examples / total_positive_examples
#total number of examples in the majority class / total number of examples in the minority class.
#18274/124=147.37
model = XGBClassifier(scale_pos_weight=147)

# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
# summarize performance
print('Mean ROC AUC: %.5f' % mean(scores))

Mean ROC AUC: 0.97334


In [20]:
clf_1 = model.fit(X_train, y_train)
pred_y_1 = clf_1.predict(X_test)

print(accuracy_score(pred_y_1, y_test))

0.9931412894375857


In [21]:
target_names = ['class 0', 'class 1']
print(classification_report(y_test, pred_y_1, target_names=target_names))

# View confusion matrix for test data and predictions
confusion_matrix(y_test, pred_y_1)

              precision    recall  f1-score   support

     class 0       0.99      1.00      1.00      1442
     class 1       0.88      0.44      0.58        16

    accuracy                           0.99      1458
   macro avg       0.93      0.72      0.79      1458
weighted avg       0.99      0.99      0.99      1458



array([[1441,    1],
       [   9,    7]])

# Tune with GridSearch CV

In [22]:
# define grid
weights = [1, 10, 25, 50, 75, 99, 100, 1000]
param_grid = dict(scale_pos_weight=weights)

In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')

In [33]:
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/Users/chathurangishyalika/opt/anaconda3/envs/tf/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:03:30] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Best: 0.931436 using {'scale_pos_weight': 1}
0.931436 (0.040774) with: {'scale_pos_weight': 1}
0.927670 (0.048305) with: {'scale_pos_weight': 10}
0.929479 (0.045071) with: {'scale_pos_weight': 25}
0.922069 (0.046362) with: {'scale_pos_weight': 50}
0.923126 (0.051366) with: {'scale_pos_weight': 75}
0.922714 (0.053929) with: {'scale_pos_weight': 99}
0.922413 (0.053952) with: {'scale_pos_weight': 100}
0.905774 (0.058887) with: {'scale_pos_weight': 1000}


In [24]:
# define model
model = XGBClassifier(scale_pos_weight=1000)

# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
# summarize performance
print('Mean ROC AUC: %.5f' % mean(scores))

Mean ROC AUC: 0.97768


In [25]:
clf_2 = model.fit(X_train, y_train)
pred_y_2 = clf_2.predict(X_test)

print(accuracy_score(pred_y_2, y_test))



0.9945130315500685


In [26]:
target_names = ['class 0', 'class 1']
print(classification_report(y_test, pred_y_2, target_names=target_names))

# View confusion matrix for test data and predictions
confusion_matrix(y_test, pred_y_2)

              precision    recall  f1-score   support

     class 0       0.99      1.00      1.00      1442
     class 1       1.00      0.50      0.67        16

    accuracy                           0.99      1458
   macro avg       1.00      0.75      0.83      1458
weighted avg       0.99      0.99      0.99      1458



array([[1442,    0],
       [   8,    8]])